In [2]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
import numpy as np

# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)


# Open Sheet
sheet = client.open_by_key("1xC719rx6N4mVk97tkXTOyu_fAFM24RFBRBXtptJH9kU")
AppLog_Sheet = sheet.worksheet("Sup Log Raw Dump")
Coverage_Sheet = sheet.worksheet("Store Coverage Raw Dump")
Attendence_Sheet = sheet.worksheet("Att Raw Dump")


# SQL Server connection details
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

# Connect and fetch into DataFrame
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = '''
               SELECT [SupervisorAppLogId]
                     ,[ManagerId]
                     ,[ManagerName]
                     ,[ManagerDesignation]
                     ,[UserId]
                     ,[UserFullName]
                     ,[UserDesignation]
                     ,[LISStoreCode]
                     ,[StoreName]
                     ,[ModuleName]
                     ,[Count]
                     ,[Time]
                 FROM [PowerBI_LISV2].[PiramalManagement].[SupervisorAppLogReport]
                 WHERE MONTH([Time])= MONTH(GETDATE()-1)
                 AND YEAR([Time])= YEAR(GETDATE()-1)

          '''

    # Fetch results into a DataFrame
    AppLog = pd.read_sql(query, conn)

    # Close the connection
    conn.close()



except Exception as e:
    print(f"❌ Error: {e}")
    

    
AppLog_Sheet.batch_clear(["A:M"])
set_with_dataframe(AppLog_Sheet, AppLog)



# Connect and fetch into DataFrame
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = '''
                SELECT *
                FROM [PowerBI_LISV2].[PiramalManagement].[ISPStoreAttendance_NormalExport]
                WHERE Month(PARSE(VisitDate AS datetime USING 'en-GB')) = MONTH(GETDATE()-1)
                AND year=YEAR(GETDATE()-1)
                AND [UserName]  LIKE '%SUP%'

          '''

    # Fetch results into a DataFrame
    Coverage = pd.read_sql(query, conn)

    # Close the connection
    conn.close()



except Exception as e:
    print(f"❌ Error: {e}")
    

    
Coverage_Sheet.batch_clear(["A:M"])
set_with_dataframe(Coverage_Sheet, Coverage)



# Connect and fetch into DataFrame
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = ''' 
               Select * from ( SELECT *
	                                ,ROW_NUMBER() OVER(PARTITION BY USERNAME
                                                             ,FullName
                                                             ,Date 
                                           ORDER BY [In-Time] desc) AS Rank,
                                           CASE WHEN  [UserName] LIKE '%SUP%' THEN 1
                                               ELSE 0  END AS SupFlag
FROM [PowerBI_LISV2].[PiramalManagement].[ISPAttendanceMasterExport]

WHERE  
       [Month]= MONTH(GETDATE()-1)
   AND [Year] = YEAR(GETDATE()-1)
   AND (
        [UserName] LIKE '%SUP%'
        OR [UserName] LIKE '%ISP%'
       ))x where Rank=1

          '''

    # Fetch results into a DataFrame
    Attendence = pd.read_sql(query, conn)

    # Close the connection
    conn.close()



except Exception as e:
    print(f"❌ Error: {e}")
    

    
Attendence_Sheet.batch_clear(["A:M"])
set_with_dataframe(Attendence_Sheet, Attendence)

✅ Connected to SQL Server


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_16756\3791251925.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AppLog = pd.read_sql(query, conn)


✅ Connected to SQL Server


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_16756\3791251925.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Coverage = pd.read_sql(query, conn)


✅ Connected to SQL Server


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_16756\3791251925.py:141: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Attendence = pd.read_sql(query, conn)
